### SocialAL Models
# Data simulation and parameter recovery - multiple subjects
KLS 8.30.19; update 3.5.22  
Project info: https://osf.io/b48n2/

Model modified from :
Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

### Python version

In [1]:
import sys
print(sys.version)  

3.7.4 (default, Aug 13 2019, 15:17:50) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


### Load modules

In [2]:
import numpy as np
import random
import math
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import itertools

## Pull in functions

In [3]:
run common_functions.ipynb

In [4]:
run priors_functions.ipynb

## Functions to simulate data

In [5]:
def select_response(prob):     
    n = random.uniform(0,1)
    if n > prob:
        response = 0
    else:
        response = 1
    return response

### Set constants

In [6]:
recip_rates = {0: 0.93, 1:0.6, 2:0.07}

### New function to simulate data for one sub

In [7]:
def sim_data(tn, params):
    # tn: number of trials desired
    # params: ground truth of parameters
    a_gain = params[0]
    a_loss = params[1]
    beta = params[2]
    prob = params[3:6]
    
    # initialize variables
    Probs = [0.5, 0.5, 0.5]
    EVs = [[9,9,9,9],[9,9,9,9],[9,9,9,9]]
    
    # generate trial sequence
    trial_sequence = np.repeat([0,1,2], tn)
    random.shuffle(trial_sequence) #print(trial_sequence)
    
    choices = []
    responses = []
    
    for x in range(0,len(trial_sequence)):
        t = trial_sequence[x] 
        
        # Make a choice
        choice = action_selection(get_action_selection_probs(beta, EVs[t]))
        choices.append(choice) 
    
        # Get a response
        recip_rate = recip_rates.get(t) 
        
        response = select_response(recip_rate) 
        responses.append(response)
    
        # after choice, update probability
        if choice != 1:
            Probs[t] = update_prob(response, Probs[t], a_gain, a_loss) 
        # then update value
        EVs[t] = update_value(Probs[t]) 

    data = {'Stim_Sequence': trial_sequence, 'Choice' : choices, 'Trustee_Response': responses}    
    return data

In [8]:
#sim_data(15,[.2,.3,2,.5, .8, .2])

### New function to sim data for multiple subjects

In [9]:
def sim_mult_subject_data(n, tn, params):
    data = pd.DataFrame(columns= ['Stim_Sequence', 'Choice', 'Trustee_Response', 'Subject', 'Alpha_gain', 'Alpha_loss','Beta', 'iProbA', 'iProbB', 'iProbC'])
    for x in range(1, n+1):
        sdt = sim_data(tn, params)
        sdt = pd.DataFrame.from_dict(sdt)
        sdt['Subject'] = [x for t in range(0,tn*3)]
        sdt['Alpha_gain'] = params[0]
        sdt['Alpha_loss'] = params[1]
        sdt['Beta'] = params[2]
        sdt['iProbA'] = params[3]
        sdt['iProbB'] = params[4]
        sdt['iProbC'] = params[5]
        data = pd.concat([data,sdt])
    return data

In [10]:
data = sim_mult_subject_data(10, 15, [.2,.3, 2, .5, .8, .2])
data

,Stim_Sequence,Choice,Trustee_Response,Subject,Alpha_gain,Alpha_loss,Beta,iProbA,iProbB,iProbC
0,1,4,1,1,0.2,0.3,2,0.5,0.8,0.2
1,1,3,0,1,0.2,0.3,2,0.5,0.8,0.2
2,1,2,1,1,0.2,0.3,2,0.5,0.8,0.2
3,2,2,0,1,0.2,0.3,2,0.5,0.8,0.2
4,1,2,0,1,0.2,0.3,2,0.5,0.8,0.2
...,...,...,...,...,...,...,...,...,...,...
40,0,4,1,10,0.2,0.3,2,0.5,0.8,0.2
41,0,4,1,10,0.2,0.3,2,0.5,0.8,0.2
42,2,1,0,10,0.2,0.3,2,0.5,0.8,0.2
43,2,1,0,10,0.2,0.3,2,0.5,0.8,0.2


### Create param space

In [11]:
# create parameter space
alpha_gains = [round(x * 0.25,1) for x in range(0,5)]
alpha_loses = [round(x * 0.25,1) for x in range(0,5)]
betas = [x * 4 for x in range(0,5)]
iprobA = [round(x * .25,1) for x in range(0,5)]
iprobB = [round(x * .25,1) for x in range(0,5)]
iprobC = [round(x * .25,1) for x in range(0,5)]

combos = list(itertools.product(alpha_gains, alpha_loses, betas, iprobA, iprobB, iprobC))
#for r in combos: print(r)

### Simulate data

In [12]:
data = pd.DataFrame(columns= ['Stim_Sequence', 'Choice', 'Trustee_Response', 'Subject', 
                              'Alpha_gain', 'Alpha_loss','Beta', 'iProbA', 'iProbB', 'iProbC'])
for p in range(len(combos)):
    dt = sim_mult_subject_data(1, 15, combos[p])
    dt['Subject'] = dt['Subject'] + p
    data = pd.concat([data, dt])
data

data.to_csv(path_or_buf = '../../output/simulation/sim_2alpha_with_priors_model_data.csv', index = False)

KeyboardInterrupt: 

### New function to fit model to multiple subjects

In [ ]:
def model_fit_mult_subject(data):
    pdt = pd.DataFrame(columns = ['Subject', 'a_gain', 'a_loss', 'beta', 'iProbA', 'iProbB', 'iProbC'])
    a = pd.unique(data['Subject'])
    for sub in range(1,len(a)+1):
        print('Subject: ', sub)
        df = data[data['Subject']==sub]
        dt = df.to_dict()
        params = model_fit(dt)
        #print('Params: ', params)
        line = {'Subject': sub, 'a_gain': params.x[0], 'a_loss': params.x[1], 'beta':params.x[2], 
                'iProbA':params.x[3], 'iProbB':params.x[4], 'iProbC':params.x[5]}
        pdt = pdt.append(line, ignore_index=True)   
    return(pdt)


In [ ]:
precover = model_fit_mult_subject(data)
precover.to_csv(path_or_buf = '../../output/simulation/sim_2alpha_with_priors_model_fit.csv', index = False)

In [ ]:
print(precover)